In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Train tensorflow or keras model on GCP or Kubeflow from Notebooks

This notebook introduces you to using Kubeflow Fairing to train the model to Kubeflow on Google Kubernetes Engine (GKE), and Google Cloud AI Platform training. This notebook demonstrate how to:
 
* Train an Keras model in a local notebook,
* Use Kubeflow Fairing to train an Keras model remotely on Kubeflow cluster,
* Use Kubeflow Fairing to train an Keras model remotely on AI Platform training,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow, and Call the deployed endpoint for predictions.

**You need Python 3.6 to use Kubeflow Fairing.**

## Setups

* Pre-conditions
    - Deployed a kubeflow cluster through https://deploy.kubeflow.cloud/
    - Have the following environment variable ready: 
        - PROJECT_ID # project host the kubeflow cluster or for running AI platform training
        - DEPLOYMENT_NAME # kubeflow deployment name, the same the cluster name after delpoyed
        - GCS_BUCKET # google cloud storage bucket

* Create service account
```bash
export SA_NAME = [service account name]
gcloud iam service-accounts create ${SA_NAME}
gcloud projects add-iam-policy-binding ${PROJECT_ID} \
    --member serviceAccount:${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com \
    --role 'roles/editor'
gcloud iam service-accounts keys create ~/key.json \
    --iam-account ${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com
```

* Authorize for Source Repository
```bash
gcloud auth configure-docker
```

* Update local kubeconfig (for submiting job to kubeflow cluster)
```bash
export CLUSTER_NAME=${DEPLOYMENT_NAME} # this is the deployment name or the kubenete cluster name
export ZONE=us-central1-c
gcloud container clusters get-credentials ${CLUSTER_NAME} --region ${ZONE}
```

* Set the environmental variable: GOOGLE_APPLICATION_CREDENTIALS
```bash
export GOOGLE_APPLICATION_CREDENTIALS = ....
```
```python
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=...
```

* Install the lastest version of fairing
```python
pip install git+https://github.com/kubeflow/fairing@master
```

**Please not that the above configuration is required for notebook service running outside Kubeflow environment. And the examples demonstrated in the notebook is fully tested on notebook service outside Kubeflow cluster also.**

**The environemt variables, e.g. service account, projects and etc, should have been pre-configured while setting up the cluster.**

In [1]:
import os
import logging
from datetime import datetime

import kubeflow.fairing as fairing
from kubeflow.fairing.cloud import gcp

In [2]:
import tensorflow as tf

In [3]:
# This is for local notebook instead of that in kubeflow cluster
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../kubeflow-pipeline/config/kubeflow-pipeline-fantasy.json'
! echo $GOOGLE_APPLICATION_CREDENTIALS

../kubeflow-pipeline/config/kubeflow-pipeline-fantasy.json


In [4]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
# For local notebook, GCP_PROJECT should be set explicitly

GCP_PROJECT = fairing.cloud.gcp.guess_project_name() # 'kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ui'
print(GCP_PROJECT)

kubeflow-pipeline-fantasy


## Define the model logic

In [5]:
def gcs_copy(src_path, dst_path):
    import subprocess
    print(subprocess.run(['gsutil', 'cp', src_path, dst_path], stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
    
def gcs_download(src_path, file_name):
    import subprocess
    print(subprocess.run(['gsutil', 'cp', src_path, file_name], stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))

In [6]:
class TensorflowModel(object):
    
    def __init__(self):
        self.model_file = "mnist_model.h5"
        self.job_dir = GCS_BUCKET
        self.model = None    
    
    def build(self):
        self.model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(512, activation=tf.nn.relu),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        ])
        self.model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        print(self.model.summary())
    
    def save_model(self):
        self.model.save(self.model_file)
        gcs_copy(self.model_file, self.job_dir + '/' + self.model_file)
    
    def train(self):
        self.build()
        
        mnist = tf.keras.datasets.mnist
        (x_train, y_train),(x_test, y_test) = mnist.load_data()
        x_train, x_test = x_train / 255.0, x_test / 255.0
        
        callbacks = [
          # Interrupt training if `val_loss` stops improving for over 2 epochs
          tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
          # Write TensorBoard logs to `./logs` directory
          tf.keras.callbacks.TensorBoard(log_dir=self.job_dir + '/logs/' 
                                         + datetime.now().date().__str__())
        ]
        self.model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
                  validation_data=(x_test, y_test))
        self.save_model()
        
    def predict(self, X):
        if not self.model:
            self.model = tf.keras.models.load_model(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)

## Train an Keras model in a notebook

In [7]:
TensorflowModel().train()

[W 200409 11:22:14 deprecation:506] From /Users/luoshixin/LocalSim/virtualPython37Fairing/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
    Instructions for updating:
    If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
None
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
   32/60000 [..............................] - ETA: 2:25 - loss: 2.3085 - acc: 0.1875

[W 200409 11:22:24 callbacks:245] Method (on_train_batch_end) is slow compared to the batch update (2.101037). Check your callbacks.


60000/60000 [==============================] - 10s 167us/sample - loss: 0.2200 - acc: 0.9359 - val_loss: 0.1246 - val_acc: 0.9617
Epoch 2/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.1000 - acc: 0.9692 - val_loss: 0.0754 - val_acc: 0.9767
Epoch 3/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.0690 - acc: 0.9780 - val_loss: 0.0658 - val_acc: 0.9779 0s - loss: 0.0693 - acc: 0.97 - ETA: 0s - loss: 0.06
Epoch 4/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.0540 - acc: 0.9824 - val_loss: 0.0633 - val_acc: 0.9809
Epoch 5/5
60000/60000 [==============================] - 6s 97us/sample - loss: 0.0428 - acc: 0.9857 - val_loss: 0.0795 - val_acc: 0.9770



## Spicify a image registry that will hold the image built by fairing

In [8]:
# In this demo, I use gsutil, therefore i compile a special image to install GoogleCloudSDK as based image
base_image = 'gcr.io/{}/kubeflow-fairing-base:latest'.format(GCP_PROJECT)
# ! gcloud builds submit --tag {base_image}


For build using docker
```bash
docker build . -t {base_image}
docker push {base_image}
```

In [9]:
BASE_IMAGE = base_image # "tensorflow/tensorflow:1.15.0-py3"
DOCKER_REGISTRY = 'gcr.io/{}/kubeflow-fairing-job-keras'.format(GCP_PROJECT)

## Deploy the training job to kubeflow cluster

In [10]:
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import KubeflowGKEBackend

train_job = TrainJob(TensorflowModel, BASE_IMAGE, input_files=["requirements.txt"],
                     docker_registry=DOCKER_REGISTRY, backend=KubeflowGKEBackend())
train_job.submit()

[W 200409 11:22:57 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
[I 200409 11:22:57 tasks:66] Building the docker image.
[I 200409 11:22:57 cluster:46] Building image using cluster builder.
[W 200409 11:22:57 base:94] /Users/luoshixin/LocalSim/virtualPython37Fairing/lib/python3.7/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200409 11:22:57 base:107] Creating docker context: /tmp/fairing_context_h3awn2kv
[W 200409 11:22:57 base:94] /Users/luoshixin/LocalSim/virtualPython37Fairing/lib/python3.7/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[W 200409 11:23:02 manager:296] Waiting for fairing-builder-nd28h-jpttp to start...
[I 200409 11:23:03 manager:302] Pod started running True


ERROR: logging before flag.Parse: E0409 03:23:07.755337       1 metadata.go:241] Failed to unmarshal scopes: json: cannot unmarshal string into Go value of type []string
INFO[0004] Resolved base name gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest to gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest
INFO[0004] Resolved base name gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest to gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest
INFO[0004] Downloading base image gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest
INFO[0004] Error while retrieving image from cache: getting file info: stat /cache/sha256:88c783033a077d26d0b09e8015ed6691d8568adcd46bd5abc470d0ad99d72e9a: no such file or directory
INFO[0004] Downloading base image gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest
INFO[0005] Built cross stage deps: map[]
INFO[0005] Downloading base image gcr.io/kubeflow-pipeline-fantasy/kubeflow-fairing-base:latest
INFO[

[W 200409 11:24:47 job:101] The job fairing-job-jp722 launched.
[W 200409 11:24:48 manager:296] Waiting for fairing-job-jp722-gvhr7 to start...
[I 200409 11:24:51 manager:302] Pod started running True


2020-04-09 03:24:51.821189: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-09 03:24:51.821369: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-09 03:24:51.821392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/app/function_shim.py", line 79, in <module>
    call(args.serialized_fn_file)
  File "/app/function_shim.py", line 61, in call
    obj = cloudpickle.load(f)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50,

  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__in

  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__in

    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3

[W 200409 11:24:58 job:173] Cleaning up job fairing-job-jp722...


'fairing-job-jp722'

## Deploy the training job as CMLE training job

Doesn’t support CMLE distributed training

In [11]:
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import GCPManagedBackend
train_job = TrainJob(TensorflowModel, BASE_IMAGE, input_files=["requirements.txt"],
                     docker_registry=DOCKER_REGISTRY, backend=GCPManagedBackend())
train_job.submit()

RuntimeError: Not able to guess the right builder for this job!

## Inspect training process with tensorboard

In [22]:
# ! tensorboard --logdir=gs://kubeflow-demo-g/logs --host=localhost --port=8777

## Deploy the trained model to Kubeflow for predictions

In [13]:
from kubeflow.fairing import PredictionEndpoint
from kubeflow.fairing.backends import KubeflowGKEBackend
# The trained_ames_model.joblib is exported during the above local training
endpoint = PredictionEndpoint(TensorflowModel, BASE_IMAGE, input_files=['mnist_model.h5', "requirements.txt"],
                              docker_registry=DOCKER_REGISTRY, backend=KubeflowGKEBackend())
endpoint.create()

Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
Using docker registry: gcr.io/gojek-kubeflow/fairing-job-tf
Using builder: <class 'fairing.builders.docker.docker.DockerBuilder'>
Building the docker image.
Building image using docker
Docker command: ['python', '/app/function_shim.py', '--serialized_fn_file', '/app/pickled_fn.p']
/Users/luoshixin/LocalSim/virtualPython36/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_ftqfzvuc
/Users/luoshixin/LocalSim/virtualPython36/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Context: /tmp/fairing_context_ftqfzvuc, Adding /Users/luoshixin/LocalSim/virtualPython36/lib/python3.6/site-packages/fairing/__init__.py at /app/fairing/__init__.py
Context: /tmp/fairing_context_ftqfzvuc, Adding /Users/luoshixin/LocalSim/virtualPython36/lib/python3.6/site-packages/fairing/runtime_config.py

Waiting for prediction endpoint to come up...


Cluster endpoint: http://35.184.251.118:5000/predict
Prediction endpoint: http://35.184.251.118:5000/predict


In [14]:
endpoint.delete()

Deleted service: kubeflow/fairing-service-vrhnq
Deleted deployment: kubeflow/fairing-deployer-fd2bz
